In [1]:
import os
import sys
proj_dir = os.path.dirname(os.path.dirname(os.path.abspath("/home/doyeonk/PM2.5-GNN-master")))
sys.path.append(proj_dir)
import shutil
import arrow
import time
import numpy as np
import xarray as xr
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy.interpolate import interp2d

# VARS = [
# '100m_u_component_of_wind',
# '100m_v_component_of_wind',
# '2m_dewpoint_temperature',
# '2m_temperature',
# 'boundary_layer_height',
# 'k_index',
# 'relative_humidity+950',
# 'relative_humidity+975',
# 'specific_humidity+950',
# 'surface_pressure',
# 'temperature+925',
# 'temperature+950',
# 'total_precipitation',
# 'u_component_of_wind+950',
# 'v_component_of_wind+950',
# 'vertical_velocity+950',
# 'vorticity+950'] 

# var_key:
# 2m_temperature: t2m
# boundary_layer_height: blh
# k_index: kx
# relative_humidity: r
# surface_pressure: sp
# total_precipitation: tp
# u_component_of_wind: u
# v_component_of_wind: v

VARS = [
'2m_temperature',
'boundary_layer_height',
'k_index',
'relative_humidity+975',
'surface_pressure',
'total_precipitation',
'u_component_of_wind+950',
'v_component_of_wind+950',
]

In [2]:
def load_pos():
    with open('/home/doyeonk/PM2.5-GNN-master/data/korea_city_idx.txt', 'r') as f: #측정소 위치 데이터(317개) -> 110개
        ls = f.readlines()

    lats, lngs = [], []
    for l in ls:
        idx, city, lng, lat = l.split(' ')
        lats.append(float(lat))
        lngs.append(float(lng))
    lats = np.array(lats)
    lngs = np.array(lngs)
    return lats, lngs


def get_nc_from_dir(dir):
    fp_list = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            if name.endswith('.nc'):
                fp = os.path.join(root, name)
                fp_list.append(fp)
    fp_list.sort()
    return fp_list


def get_key_from_file(fp):
    var = os.path.basename(fp).split('@')[0].split('+')[0]
    key = config['era5']['var_key'][var]
    return var, key


def plot_test_image():
    fp_list = get_nc_from_dir(config['era5']['raw_test_dir'])

    for fp in fp_list:
        _, key = get_key_from_file(fp)
        ds = xr.open_dataset(fp, engine='netcdf4')
        var_data = ds[key].values
        plt.imshow(var_data[0])
        plt.show()

def load_var(v):
    print('load', v)
    years = ['2015', '2016', '2017']
    months = ['%02d' % i for i in range(1,13)]
    DIR = '/home/doyeonk/PM2.5-GNN-master/data/ERA_data_1517/'
    data = [] 
    ts = []
    for y in years: 
        for m in months:
            fn = '%s@%s@%s.nc' % (v, y, m) 
            ds = xr.open_dataset(DIR+fn, engine='netcdf4')
            v_name = list(ds.data_vars.variables.mapping.keys())[0]
            data.append(np.array(ds[v_name]))
            ts.append(np.array(ds.time))
            lng, lat = np.array(ds.longitude), np.array(ds.latitude)
    data = np.concatenate(data, axis=0)
    ts = np.concatenate(ts, axis=0)
    print('data', data.shape)
    print('ts', ts.shape, ts[-1])
    ts = (ts - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's') 
    print('load', v, 'done')
    return data, ts, lng, lat

def interp(dts, lats, lngs, v):
    data, ts, lng, lat = load_var(v)
    #lng = np.broadcast_to(lng, (data[0].shape)).flatten()
    #lat = np.broadcast_to(lat.reshape((lat.shape[0], 1)), (data[0].shape)).flatten()
    vs = [] 
    #lngs = lngs[:5]
    #lats = lats[:5]
    for dt in dts:
        idx = np.where(ts==dt)[0][0]
        func = interp2d(lng, lat, data[idx])    
        #interp_data = func(lngs, lats)[::-1].diagonal()[::-1] 
        #interp_data2 = func(lngs, lats)
        interp_data = []
        for _lng, _lat in zip(lngs, lats):
            interp_data.append(func(_lng, _lat))
        #print(interp_data)
        #print(interp_data2)
        #for i, j in zip(interp_data,interp_data2):
        #    print('eq', i, j)
        #exit()
        vs.append(interp_data)
    vs = np.array(vs)
    print('interp_data', vs.shape)
    return vs

def main():
    #lats, lngs = np.array([39.8572]), np.array([116.2803])
    lats, lngs = load_pos()
    start = arrow.get('2015/01/01', 'YYYY/MM/DD').timestamp
    end = arrow.get('2017/12/31', 'YYYY/MM/DD').timestamp
   
    from datetime import datetime
    dts = range(start, end, 3600*3)
    print(len(dts))
    print(datetime.fromtimestamp(dts[-1]))
    data = []
    vs = []
    print(lats.shape,lngs.shape)
    for v in VARS:
        print('interp', v)
        t0 = time.time()
        values = interp(dts,lats,lngs,v)
        print('interp done', v, time.time()-t0)
        data.append(values)
    data = np.concatenate(data, axis=2)
    print(data.shape)
    np.savez('./data/metero_all_1517.npz', x=data) #dataset
    return

if __name__ == '__main__':
    main()

8760
2017-12-31 06:00:00
(101,) (101,)
interp 2m_temperature
load 2m_temperature
data (26304, 49, 65)
ts (26304,) 2017-12-31T23:00:00.000000000
load 2m_temperature done


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


interp_data (8760, 101, 1)
interp done 2m_temperature 23.783031225204468
interp boundary_layer_height
load boundary_layer_height
data (26304, 49, 65)
ts (26304,) 2017-12-31T23:00:00.000000000
load boundary_layer_height done


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


interp_data (8760, 101, 1)
interp done boundary_layer_height 23.435126543045044
interp k_index
load k_index
data (26304, 49, 65)
ts (26304,) 2017-12-31T23:00:00.000000000
load k_index done


...

interp_data (8760, 101, 1)
interp done total_precipitation 23.588659048080444
interp u_component_of_wind+950
load u_component_of_wind+950
data (26304, 49, 65)
ts (26304,) 2017-12-31T23:00:00.000000000
load u_component_of_wind+950 done


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


interp_data (8760, 101, 1)
interp done u_component_of_wind+950 23.582762956619263
interp v_component_of_wind+950
load v_component_of_wind+950
data (26304, 49, 65)
ts (26304,) 2017-12-31T23:00:00.000000000
load v_component_of_wind+950 done


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


interp_data (8760, 101, 1)
interp done v_component_of_wind+950 23.42660689353943
(8760, 101, 8)


In [34]:
import os
import sys
proj_dir = os.path.dirname(os.path.dirname(os.path.abspath("/home/doyeonk/PM2.5-GNN-master")))
sys.path.append(proj_dir)
import shutil
import arrow
import time
import numpy as np
import xarray as xr
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy.interpolate import interp2d


# VARS = [
# '100m_u_component_of_wind',
# '100m_v_component_of_wind',
# '2m_dewpoint_temperature',
# '2m_temperature',
# 'boundary_layer_height',
# 'k_index',
# 'relative_humidity+950',
# 'relative_humidity+975',
# 'specific_humidity+950',
# 'surface_pressure',
# 'temperature+925',
# 'temperature+950',
# 'total_precipitation',
# 'u_component_of_wind+950',
# 'v_component_of_wind+950',
# 'vertical_velocity+950',
# 'vorticity+950',]

VARS = [
'2m_temperature',
'boundary_layer_height',
'k_index',
'relative_humidity+975',
'surface_pressure',
'total_precipitation',
'u_component_of_wind+950',
'v_component_of_wind+950',
]


def load_pos():
    with open('/home/doyeonk/PM2.5-GNN-master/data/korea_city_idx.txt', 'r') as f:
        ls = f.readlines()

    lats, lngs = [], []
    for l in ls:
        idx, city, lng, lat = l.split(' ')
        lats.append(float(lat))
        lngs.append(float(lng))
    lats = np.array(lats)
    lngs = np.array(lngs)
    return lats, lngs


def get_nc_from_dir(dir):
    fp_list = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            if name.endswith('.nc'):
                fp = os.path.join(root, name)
                fp_list.append(fp)
    fp_list.sort()
    return fp_list


def get_key_from_file(fp):
    var = os.path.basename(fp).split('@')[0].split('+')[0]
    key = config['era5']['var_key'][var]
    return var, key


def plot_test_image():
    fp_list = get_nc_from_dir(config['era5']['raw_test_dir'])

    for fp in fp_list:
        _, key = get_key_from_file(fp)
        ds = xr.open_dataset(fp, engine='netcdf4')
        var_data = ds[key].values
        plt.imshow(var_data[0])
        plt.show()

def load_var(v):
    print('load', v)
    years = ['2015', '2016', '2017']
    months = ['%02d' % i for i in range(1,13)]
    DIR = '/home/doyeonk/PM2.5-GNN-master/data/ERA_data_1517/'
    data = [] 
    ts = []
    for y in years: 
        for m in months:
            fn = '%s@%s@%s.nc' % (v, y, m) 
            ds = xr.open_dataset(DIR+fn, engine='netcdf4')
            v_name = list(ds.data_vars.variables.mapping.keys())[0]
            data.append(np.array(ds[v_name]))
            ts.append(np.array(ds.time))
            lng, lat = np.array(ds.longitude), np.array(ds.latitude)
    data = np.concatenate(data, axis=0)
    ts = np.concatenate(ts, axis=0)
    print('data', data.shape)
    print('ts', ts.shape, ts[-1])
    ts = (ts - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's') 
    print('load', v, 'done')
    return data, ts, lng, lat

def interp(dts, lats, lngs, v):
    data, ts, lng, lat = load_var(v)
    #lng = np.broadcast_to(lng, (data[0].shape)).flatten()
    #lat = np.broadcast_to(lat.reshape((lat.shape[0], 1)), (data[0].shape)).flatten()
    vs = [] 
    #lngs = lngs[:5]
    #lats = lats[:5]
    for dt in dts:
        idx = np.where(ts==dt)[0][0]
        func = interp2d(lng, lat, data[idx])    
        #interp_data = func(lngs, lats)[::-1].diagonal()[::-1] 
        #interp_data2 = func(lngs, lats)
        interp_data = []
        for _lng, _lat in zip(lngs, lats):
            interp_data.append(func(_lng, _lat))
        #print(interp_data)
        #print(interp_data2)
        #for i, j in zip(interp_data,interp_data2):
        #    print('eq', i, j)
        #exit()
        vs.append(interp_data)
    vs = np.array(vs)
    print('interp_data', vs.shape)
    return vs


def main():
    #lats, lngs = np.array([39.8572]), np.array([116.2803])
    lats, lngs = load_pos()
    start = arrow.get('2015/01/01', 'YYYY/MM/DD').timestamp
    end = arrow.get('2017/12/31', 'YYYY/MM/DD').timestamp
    from datetime import datetime
    dts = range(start, end, 3600*3)
    print(len(dts))
    print(datetime.fromtimestamp(dts[-1]))
    data = []
    vs = []
    print(lats.shape,lngs.shape)
    for v in VARS:
        print('interp', v)
        t0 = time.time()
        values = interp(dts,lats,lngs,v)
        print('interp done', v, time.time()-t0)
        data.append(values)
    data = np.concatenate(data, axis=2)
    print(data.shape)
    np.savez('./data/metero_all_1517.npz', x=data)
    return


if __name__ == '__main__':
    main()


8760
2017-12-31 06:00:00
(317,) (317,)
interp 2m_temperature
load 2m_temperature
data (26304, 49, 65)
ts (26304,) 2017-12-31T23:00:00.000000000
load 2m_temperature done


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


interp_data (8760, 317, 1)
interp done 2m_temperature 70.96327829360962
interp boundary_layer_height
load boundary_layer_height
data (26304, 49, 65)
ts (26304,) 2017-12-31T23:00:00.000000000
load boundary_layer_height done


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


interp_data (8760, 317, 1)
interp done boundary_layer_height 70.08152890205383
interp k_index
load k_index
data (26304, 49, 65)
ts (26304,) 2017-12-31T23:00:00.000000000
load k_index done


...

interp_data (8760, 317, 1)
interp done total_precipitation 66.68016910552979
interp u_component_of_wind+950
load u_component_of_wind+950
data (26304, 49, 65)
ts (26304,) 2017-12-31T23:00:00.000000000
load u_component_of_wind+950 done


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


interp_data (8760, 317, 1)
interp done u_component_of_wind+950 66.61903858184814
interp v_component_of_wind+950
load v_component_of_wind+950
data (26304, 49, 65)
ts (26304,) 2017-12-31T23:00:00.000000000
load v_component_of_wind+950 done


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


interp_data (8760, 317, 1)
interp done v_component_of_wind+950 67.64628648757935
(8760, 317, 8)


In [3]:
# 인풋데이터 확인
feature_fp = '/home/doyeonk/PM2.5-GNN-master/data/metero_all_1517.npz'

feature = np.load(feature_fp)['x']
feature.shape

(8760, 101, 8)